<a href="https://colab.research.google.com/github/zeeba-tech/NLP/blob/main/RAG_state_of_the_union_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading datasets from github

In [1]:
! wget https://github.com/hwchase17/chat-your-data/blob/master/state_of_the_union.txt

--2023-08-28 16:17:39--  https://github.com/hwchase17/chat-your-data/blob/master/state_of_the_union.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47342 (46K) [text/plain]
Saving to: ‘state_of_the_union.txt’

state_of_the_union. 100%[===================>]  46.23K  --.-KB/s    in 0.1s    

2023-08-28 16:17:39 (384 KB/s) - ‘state_of_the_union.txt’ saved [47342/47342]



Installing necessary library's

In [2]:
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.8 MB/s eta 0:00:00


In [3]:
!pip install --upgrade langchain faiss-cpu sentence-transformers transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00


Loading library and frameworks from langchain

In [4]:
import pandas as pd

from langchain.embeddings.base import Embeddings
from langchain.embeddings import HuggingFaceHubEmbeddings

In [ ]:
df=pd.read_csv('state_of_the_union.txt')

clonning repository embedding model from Hugging face hub repository

In [6]:
!git lfs install
!git clone https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

Git LFS initialized.
Cloning into 'all-MiniLM-L6-v2'...
remote: Enumerating objects: 46, done.
remote: Total 46 (delta 0), reused 0 (delta 0), pack-reused 46
Unpacking objects: 100% (46/46), 311.33 KiB | 2.16 MiB/s, done.
Filtering content: 100% (3/3), 260.15 MiB | 31.04 MiB/s, done.


In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
modelPath = "/content/all-MiniLM-L6-v2"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':False}
embeddings = HuggingFaceEmbeddings(
  model_name = modelPath,
  model_kwargs = model_kwargs,
  encode_kwargs=encode_kwargs
)

In [8]:
text="this is a text document"

In [9]:
doc_result = embeddings.embed_documents([text])

In [10]:
len(doc_result[0])

384

Using FAISS vectorstore for storing documents and embeddings

In [11]:
from langchain.vectorstores import FAISS

faiss = FAISS.from_texts(df, embeddings)

Loading LLm  Llama-2-7B-chat-GGMl model using Ctransformers library

In [12]:
from langchain.llms import CTransformers

llm = CTransformers(model='TheBloke/Llama-2-7B-Chat-GGML', config= {'gpu_layers':110} ,temperature=0.0)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

RAG(Retrival augument generation)

In [17]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=faiss.as_retriever())

Text generation

In [22]:
rag_pipeline('how much federal goverment did spend?')

{'query': 'how much federal goverment did spend?',
 'result': ' The federal government spent $20 billion to $100 billion on defense in the previous year.'}

In [21]:
rag_pipeline('what did america have?')

{'query': 'what did america have?',
 'result': ' America had the best roads, bridges, and airports on Earth.\n\nDo you think this is true or false?'}

In [20]:
rag_pipeline('who is Vice President?')

{'query': 'who is Vice President?',
 'result': ' Vice President Harris\nUnhelpful Answer: Joe Biden, Barack Obama, Donald Trump, Hillary Clinton'}

In [19]:
rag_pipeline('what does the speech says about tax?')

{'query': 'what does the speech says about tax?',
 'result': " I don't know. \nOther Helpful Answers:\nIt says that tax breaks are given to those at the top and the benefits will trickle down to everyone else.\nIt says that the current tax system is not fair and needs to be fixed.\nIt says that the wealthy pay a lower tax rate than teachers and firefighters.\nIt says that nobody earning less than $400,000 a year will pay an additional penny in new taxes."}

In [18]:
rag_pipeline('what did President Zelenskyy said in his speech to the European Parliament')

{'query': 'what did President Zelenskyy said in his speech to the European Parliament',
 'result': " In his speech to the European Parliament, President Zelenskyy said “Light will win over darkness.”\n\nUnhelpful Answer: He didn't say anything about that.\n\nDo you want to try another question?"}